# Tennis Match Upset Features Engineering

This notebook builds upon our cleaned_engineered_data_atp_matches to investigate what features influence tennis match upsets. We'll create and analyze various derived features that capture different aspects of player performance and match dynamics.

## Feature Categories

1. **Recent Form**
   - 3-month rolling win-loss ratio
   - 6-month rolling win-loss ratio
   - 12-month rolling win-loss ratio

2. **Surface Specialization**
   - Surface-specific win rates
   - Performance differentials across surfaces
   - Surface adaptation indicators

3. **Head-to-Head Dynamics**
   - Historical matchup statistics
   - Win percentages against specific opponents
   - Performance in previous meetings

## 1. Import Libraries and Load Data

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("Libraries imported successfully!")

Libraries imported successfully!


In [6]:
# Define paths
data_folder = r'c:\poly_code\COMP. DATA ANALYSIS PROJECT\Cleaned_engineered_Data_Atp_matches'
original_folder = r'c:\poly_code\COMP. DATA ANALYSIS PROJECT\Atp_Matches_Folder'

# Load all years of data and merge with original to get dates
years = [2016, 2017, 2018, 2019, 2021, 2022, 2023, 2024]
dfs = {}

for year in years:
    # Load cleaned/engineered data
    file_path = os.path.join(data_folder, f'merged_matches_{year}.xlsx')
    cleaned_df = pd.read_excel(file_path)
    
    # Load original ATP data to get tourney_date
    original_path = os.path.join(original_folder, f'atp_matches_{year}.csv')
    original_df = pd.read_csv(original_path)
    
    # Merge to get tourney_date (use tourney_id and winner_name as keys)
    merged_df = cleaned_df.merge(
        original_df[['tourney_id', 'winner_name', 'loser_name', 'tourney_date']],
        on=['tourney_id', 'winner_name', 'loser_name'],
        how='left'
    )
    
    dfs[year] = merged_df
    print(f"Loaded {year}: {len(dfs[year])} matches")

# Combine all data for feature engineering
all_data = pd.concat(dfs.values(), ignore_index=True)

# Convert tourney_date to datetime
all_data['tourney_date'] = pd.to_datetime(all_data['tourney_date'], format='%Y%m%d')

print(f"\nTotal matches across all years: {len(all_data)}")
print(f"Date range: {all_data['tourney_date'].min()} to {all_data['tourney_date'].max()}")

Loaded 2016: 2812 matches
Loaded 2017: 2744 matches
Loaded 2018: 2784 matches
Loaded 2019: 2633 matches
Loaded 2021: 2573 matches
Loaded 2022: 2558 matches
Loaded 2023: 2668 matches
Loaded 2024: 2754 matches

Total matches across all years: 21526
Date range: 2016-01-04 00:00:00 to 2024-12-18 00:00:00


In [7]:
# Check columns and data structure
print("Columns in dataset:")
print(all_data.columns.tolist())
print(f"\nData shape: {all_data.shape}")
print("\nFirst few rows:")
all_data.head()

Columns in dataset:
['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level', 'winner_seed', 'winner_name', 'winner_ht', 'winner_ioc', 'winner_age', 'loser_seed', 'loser_name', 'loser_ht', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round', 'minutes', 'rank_difference', 'upset', 'w_ace_df_ratio', 'l_ace_df_ratio', 'w_1st_serve_pct', 'l_1st_serve_pct', 'w_1st_serve_won_pct', 'l_1st_serve_won_pct', 'w_2nd_serve_won_pct', 'l_2nd_serve_won_pct', 'w_bp_saved_pct', 'l_bp_saved_pct', 'w_serve_efficiency', 'l_serve_efficiency', 'tourney_date']

Data shape: (21526, 34)

First few rows:


,tourney_id,tourney_name,surface,draw_size,tourney_level,winner_seed,winner_name,winner_ht,winner_ioc,winner_age,loser_seed,loser_name,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,rank_difference,upset,w_ace_df_ratio,l_ace_df_ratio,w_1st_serve_pct,l_1st_serve_pct,w_1st_serve_won_pct,l_1st_serve_won_pct,w_2nd_serve_won_pct,l_2nd_serve_won_pct,w_bp_saved_pct,l_bp_saved_pct,w_serve_efficiency,l_serve_efficiency,tourney_date
0,2016-M020,Brisbane,Hard,32,A,NaN,Mikhail Kukushkin,183.0,KAZ,28.0,NaN,Denis Istomin,185.0,UZB,29.3,6-2 7-5,3,R32,84,4.0,0,0.333333,2.137537,0.537313,0.603774,0.750000,0.687500,0.645161,0.571429,1.000000,0.571429,0.161290,0.224490,2016-01-04
1,2016-M020,Brisbane,Hard,32,A,NaN,Radek Stepanek,185.0,CZE,37.1,NaN,Dusan Lajovic,183.0,SRB,25.5,6-0 6-3,3,R32,67,121.0,1,1.500000,0.000000,0.520833,0.543478,0.720000,0.600000,0.695652,0.380952,1.000000,0.500000,0.751304,0.000000,2016-01-04
2,2016-M020,Brisbane,Hard,32,A,7.0,Bernard Tomic,196.0,AUS,23.2,NaN,Nicolas Mahut,191.0,FRA,33.9,6-4 6-3,3,R32,69,-53.0,0,3.410724,4.000000,0.576271,0.580000,0.823529,0.724138,0.560000,0.476190,0.800000,0.500000,0.368941,0.689655,2016-01-04
3,2016-M020,Brisbane,Hard,32,A,NaN,Ivan Dodig,183.0,CRO,31.0,NaN,Oliver Anderson,175.0,AUS,17.6,6-3 6-2,3,R32,67,-726.0,0,5.500000,3.000000,0.612245,0.576923,0.800000,0.733333,0.684211,0.409091,0.666188,0.500000,3.010526,0.450000,2016-01-04
4,2016-M020,Brisbane,Hard,32,A,NaN,Lucas Pouille,185.0,FRA,21.8,NaN,Yoshihito Nishioka,170.0,JPN,20.2,4-6 6-3 7-5,3,R32,143,-39.0,0,8.500000,1.333333,0.673684,0.533333,0.828125,0.656250,0.483871,0.535714,0.500000,0.800000,1.702999,0.375000,2016-01-04


## 2. Recent Form Features

Calculate rolling win-loss ratios over different time windows (3, 6, and 12 months) for both winner and loser.

In [8]:
def calculate_recent_form(df, player_col, date_col='tourney_date', windows=[90, 180, 365]):
    """
    Calculate rolling win-loss ratio for a player over different time windows.
    
    Parameters:
    - df: DataFrame with match data
    - player_col: Column name for the player ('winner_name' or 'loser_name')
    - date_col: Column name for the date
    - windows: List of days for rolling windows (e.g., 90 days = ~3 months)
    
    Returns:
    - DataFrame with rolling statistics
    """
    # Convert date to datetime if needed
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    
    # Sort by player and date
    df = df.sort_values([player_col, date_col])
    
    results = []
    
    for player in df[player_col].unique():
        player_matches = df[df[player_col] == player].copy()
        
        for idx, row in player_matches.iterrows():
            current_date = row[date_col]
            record = {'player': player, 'date': current_date, 'match_idx': idx}
            
            for window in windows:
                # Get matches within the window BEFORE current date
                window_start = current_date - timedelta(days=window)
                
                # Matches where player won
                wins = len(df[(df['winner_name'] == player) & 
                             (df[date_col] >= window_start) & 
                             (df[date_col] < current_date)])
                
                # Matches where player lost
                losses = len(df[(df['loser_name'] == player) & 
                               (df[date_col] >= window_start) & 
                               (df[date_col] < current_date)])
                
                total = wins + losses
                
                # Calculate win ratio
                if total > 0:
                    win_ratio = wins / total
                else:
                    win_ratio = np.nan
                
                # Store results
                record[f'wins_{window}d'] = wins
                record[f'losses_{window}d'] = losses
                record[f'total_matches_{window}d'] = total
                record[f'win_ratio_{window}d'] = win_ratio
            
            results.append(record)
    
    return pd.DataFrame(results)

print("Recent form function defined!")

Recent form function defined!


In [9]:
# Calculate recent form for winners
print("Calculating recent form for winners...")
winner_form = calculate_recent_form(all_data, 'winner_name')

# Rename columns to indicate these are for winners
winner_form_cols = {col: f'winner_{col}' if col not in ['player', 'date', 'match_idx'] else col 
                    for col in winner_form.columns}
winner_form = winner_form.rename(columns=winner_form_cols)

print(f"Winner form calculated: {len(winner_form)} records")
winner_form.head()

Calculating recent form for winners...
Winner form calculated: 21526 records


,player,date,match_idx,winner_wins_90d,winner_losses_90d,winner_total_matches_90d,winner_win_ratio_90d,winner_wins_180d,winner_losses_180d,winner_total_matches_180d,winner_win_ratio_180d,winner_wins_365d,winner_losses_365d,winner_total_matches_365d,winner_win_ratio_365d
0,Abedallah Shelbayh,2023-04-17,17100,0,1,1,0.0,0,1,1,0.00,0,1,1,0.000000
1,Abedallah Shelbayh,2023-11-06,18707,0,0,0,NaN,0,2,2,0.00,1,5,6,0.166667
2,Abedallah Shelbayh,2023-11-27,18770,1,1,2,0.5,1,3,4,0.25,2,6,8,0.250000
3,Adam Pavlasek,2016-05-23,1339,0,0,0,NaN,0,0,0,NaN,0,0,0,NaN
4,Adam Pavlasek,2016-07-18,1877,1,1,2,0.5,1,1,2,0.50,1,1,2,0.500000


In [10]:
# Calculate recent form for losers
print("Calculating recent form for losers...")
loser_form = calculate_recent_form(all_data, 'loser_name')

# Rename columns to indicate these are for losers
loser_form_cols = {col: f'loser_{col}' if col not in ['player', 'date', 'match_idx'] else col 
                   for col in loser_form.columns}
loser_form = loser_form.rename(columns=loser_form_cols)

print(f"Loser form calculated: {len(loser_form)} records")
loser_form.head()

Calculating recent form for losers...
Loser form calculated: 21526 records


,player,date,match_idx,loser_wins_90d,loser_losses_90d,loser_total_matches_90d,loser_win_ratio_90d,loser_wins_180d,loser_losses_180d,loser_total_matches_180d,loser_win_ratio_180d,loser_wins_365d,loser_losses_365d,loser_total_matches_365d,loser_win_ratio_365d
0,Abdulhamid Mubarak,2016-03-04,641,0,0,0,NaN,0,0,0,NaN,0,0,0,NaN
1,Abdullah Maqdas,2017-02-03,3108,0,0,0,NaN,0,0,0,NaN,0,0,0,NaN
2,Abdullah Maqdas,2017-02-03,3110,0,0,0,NaN,0,0,0,NaN,0,0,0,NaN
3,Abdulrahman Alawadhi,2016-03-04,640,0,0,0,NaN,0,0,0,NaN,0,0,0,NaN
4,Abedallah Shelbayh,2023-02-20,16561,0,0,0,NaN,0,0,0,NaN,0,0,0,NaN


## 3. Surface Specialization Features

Calculate surface-specific win rates and performance differentials for each player.

In [11]:
def calculate_surface_stats(df, player_col, date_col='tourney_date', surface_col='surface'):
    """
    Calculate surface-specific statistics for each player.
    
    Returns surface win rates and performance differentials.
    """
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.sort_values([player_col, date_col])
    
    results = []
    
    for player in df[player_col].unique():
        player_matches = df[df[player_col] == player].copy()
        
        for idx, row in player_matches.iterrows():
            current_date = row[date_col]
            current_surface = row[surface_col]
            
            record = {
                'player': player, 
                'date': current_date, 
                'match_idx': idx,
                'current_surface': current_surface
            }
            
            # Get all matches BEFORE current date
            wins_all = df[(df['winner_name'] == player) & (df[date_col] < current_date)]
            losses_all = df[(df['loser_name'] == player) & (df[date_col] < current_date)]
            
            # Overall statistics
            total_wins = len(wins_all)
            total_losses = len(losses_all)
            total_matches = total_wins + total_losses
            
            record['career_matches'] = total_matches
            record['career_wins'] = total_wins
            record['career_win_rate'] = total_wins / total_matches if total_matches > 0 else np.nan
            
            # Calculate stats for each surface
            surfaces = ['Hard', 'Clay', 'Grass', 'Carpet']
            
            for surface in surfaces:
                # Wins on this surface
                surface_wins = len(wins_all[wins_all[surface_col] == surface])
                surface_losses = len(losses_all[losses_all[surface_col] == surface])
                surface_matches = surface_wins + surface_losses
                
                # Win rate on this surface
                if surface_matches > 0:
                    surface_win_rate = surface_wins / surface_matches
                else:
                    surface_win_rate = np.nan
                
                record[f'{surface.lower()}_matches'] = surface_matches
                record[f'{surface.lower()}_wins'] = surface_wins
                record[f'{surface.lower()}_win_rate'] = surface_win_rate
                
                # Surface differential (compared to overall win rate)
                if total_matches > 0 and surface_matches > 0:
                    overall_rate = total_wins / total_matches
                    record[f'{surface.lower()}_differential'] = surface_win_rate - overall_rate
                else:
                    record[f'{surface.lower()}_differential'] = np.nan
            
            # Identify best and worst surfaces
            surface_rates = {}
            for surface in surfaces:
                rate = record.get(f'{surface.lower()}_win_rate')
                matches = record.get(f'{surface.lower()}_matches', 0)
                if not np.isnan(rate) and matches >= 3:  # Require at least 3 matches
                    surface_rates[surface] = rate
            
            if surface_rates:
                record['best_surface'] = max(surface_rates, key=surface_rates.get)
                record['worst_surface'] = min(surface_rates, key=surface_rates.get)
                record['best_surface_win_rate'] = surface_rates[record['best_surface']]
                record['worst_surface_win_rate'] = surface_rates[record['worst_surface']]
                record['surface_versatility'] = (record['best_surface_win_rate'] - 
                                                 record['worst_surface_win_rate'])
            else:
                record['best_surface'] = np.nan
                record['worst_surface'] = np.nan
                record['best_surface_win_rate'] = np.nan
                record['worst_surface_win_rate'] = np.nan
                record['surface_versatility'] = np.nan
            
            results.append(record)
    
    return pd.DataFrame(results)

print("Surface specialization function defined!")

Surface specialization function defined!


In [12]:
# Calculate surface stats for winners
print("Calculating surface statistics for winners...")
winner_surface = calculate_surface_stats(all_data, 'winner_name')

# Rename columns
winner_surface_cols = {col: f'winner_{col}' if col not in ['player', 'date', 'match_idx'] else col 
                       for col in winner_surface.columns}
winner_surface = winner_surface.rename(columns=winner_surface_cols)

print(f"Winner surface stats calculated: {len(winner_surface)} records")
winner_surface.head()

Calculating surface statistics for winners...
Winner surface stats calculated: 21526 records


,player,date,match_idx,winner_current_surface,winner_career_matches,winner_career_wins,winner_career_win_rate,winner_hard_matches,winner_hard_wins,winner_hard_win_rate,winner_hard_differential,winner_clay_matches,winner_clay_wins,winner_clay_win_rate,winner_clay_differential,winner_grass_matches,winner_grass_wins,winner_grass_win_rate,winner_grass_differential,winner_carpet_matches,winner_carpet_wins,winner_carpet_win_rate,winner_carpet_differential,winner_best_surface,winner_worst_surface,winner_best_surface_win_rate,winner_worst_surface_win_rate,winner_surface_versatility
0,Abedallah Shelbayh,2023-04-17,17100,Clay,1,0,0.000000,1,0,0.000000,0.000000,0,0,NaN,NaN,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Abedallah Shelbayh,2023-11-06,18707,Hard,6,1,0.166667,1,0,0.000000,-0.166667,3,1,0.333333,0.166667,2,0,0.0,-0.166667,0,0,NaN,NaN,Clay,Clay,0.333333,0.333333,0.0
2,Abedallah Shelbayh,2023-11-27,18770,Hard,8,2,0.250000,3,1,0.333333,0.083333,3,1,0.333333,0.083333,2,0,0.0,-0.250000,0,0,NaN,NaN,Hard,Hard,0.333333,0.333333,0.0
3,Adam Pavlasek,2016-05-23,1339,Clay,0,0,NaN,0,0,NaN,NaN,0,0,NaN,NaN,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Adam Pavlasek,2016-07-18,1877,Clay,2,1,0.500000,0,0,NaN,NaN,2,1,0.500000,0.000000,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Calculate surface stats for losers
print("Calculating surface statistics for losers...")
loser_surface = calculate_surface_stats(all_data, 'loser_name')

# Rename columns
loser_surface_cols = {col: f'loser_{col}' if col not in ['player', 'date', 'match_idx'] else col 
                      for col in loser_surface.columns}
loser_surface = loser_surface.rename(columns=loser_surface_cols)

print(f"Loser surface stats calculated: {len(loser_surface)} records")
loser_surface.head()

Calculating surface statistics for losers...
Loser surface stats calculated: 21526 records


,player,date,match_idx,loser_current_surface,loser_career_matches,loser_career_wins,loser_career_win_rate,loser_hard_matches,loser_hard_wins,loser_hard_win_rate,loser_hard_differential,loser_clay_matches,loser_clay_wins,loser_clay_win_rate,loser_clay_differential,loser_grass_matches,loser_grass_wins,loser_grass_win_rate,loser_grass_differential,loser_carpet_matches,loser_carpet_wins,loser_carpet_win_rate,loser_carpet_differential,loser_best_surface,loser_worst_surface,loser_best_surface_win_rate,loser_worst_surface_win_rate,loser_surface_versatility
0,Abdulhamid Mubarak,2016-03-04,641,Clay,0,0,NaN,0,0,NaN,NaN,0,0,NaN,NaN,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Abdullah Maqdas,2017-02-03,3108,Hard,0,0,NaN,0,0,NaN,NaN,0,0,NaN,NaN,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Abdullah Maqdas,2017-02-03,3110,Hard,0,0,NaN,0,0,NaN,NaN,0,0,NaN,NaN,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Abdulrahman Alawadhi,2016-03-04,640,Clay,0,0,NaN,0,0,NaN,NaN,0,0,NaN,NaN,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Abedallah Shelbayh,2023-02-20,16561,Hard,0,0,NaN,0,0,NaN,NaN,0,0,NaN,NaN,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 4. Head-to-Head Dynamics Features

Calculate historical matchup statistics between specific opponents.

In [14]:
def calculate_h2h_stats(df, date_col='tourney_date'):
    """
    Calculate head-to-head statistics between players.
    
    For each match, calculate:
    - Previous meetings between the two players
    - Winner's record against loser
    - Last meeting result
    - H2H win percentage
    """
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.sort_values(date_col)
    
    results = []
    
    for idx, row in df.iterrows():
        current_date = row[date_col]
        winner = row['winner_name']
        loser = row['loser_name']
        
        record = {
            'match_idx': idx,
            'date': current_date,
            'winner': winner,
            'loser': loser
        }
        
        # Find all previous matches between these two players
        previous_matches = df[
            (df[date_col] < current_date) &
            (
                ((df['winner_name'] == winner) & (df['loser_name'] == loser)) |
                ((df['winner_name'] == loser) & (df['loser_name'] == winner))
            )
        ]
        
        # Total previous meetings
        record['h2h_total_meetings'] = len(previous_matches)
        
        if len(previous_matches) > 0:
            # Winner's wins against loser
            winner_wins = len(previous_matches[previous_matches['winner_name'] == winner])
            record['h2h_winner_wins'] = winner_wins
            record['h2h_loser_wins'] = len(previous_matches) - winner_wins
            record['h2h_winner_win_pct'] = winner_wins / len(previous_matches)
            
            # Last meeting information
            last_meeting = previous_matches.iloc[-1]
            record['h2h_last_meeting_date'] = last_meeting[date_col]
            record['h2h_days_since_last_meeting'] = (current_date - last_meeting[date_col]).days
            record['h2h_last_meeting_winner_won'] = 1 if last_meeting['winner_name'] == winner else 0
            
            # Surface-specific H2H if available
            current_surface = row['surface']
            surface_h2h = previous_matches[previous_matches['surface'] == current_surface]
            
            if len(surface_h2h) > 0:
                surface_winner_wins = len(surface_h2h[surface_h2h['winner_name'] == winner])
                record['h2h_surface_meetings'] = len(surface_h2h)
                record['h2h_surface_winner_wins'] = surface_winner_wins
                record['h2h_surface_winner_win_pct'] = surface_winner_wins / len(surface_h2h)
            else:
                record['h2h_surface_meetings'] = 0
                record['h2h_surface_winner_wins'] = 0
                record['h2h_surface_winner_win_pct'] = np.nan
            
            # Momentum: won last 2 meetings?
            if len(previous_matches) >= 2:
                last_two = previous_matches.iloc[-2:]
                record['h2h_winner_won_last_2'] = int(
                    all(match['winner_name'] == winner for _, match in last_two.iterrows())
                )
            else:
                record['h2h_winner_won_last_2'] = np.nan
                
        else:
            # First meeting
            record['h2h_winner_wins'] = 0
            record['h2h_loser_wins'] = 0
            record['h2h_winner_win_pct'] = np.nan
            record['h2h_last_meeting_date'] = pd.NaT
            record['h2h_days_since_last_meeting'] = np.nan
            record['h2h_last_meeting_winner_won'] = np.nan
            record['h2h_surface_meetings'] = 0
            record['h2h_surface_winner_wins'] = 0
            record['h2h_surface_winner_win_pct'] = np.nan
            record['h2h_winner_won_last_2'] = np.nan
        
        results.append(record)
    
    return pd.DataFrame(results)

print("Head-to-head function defined!")

Head-to-head function defined!


In [15]:
# Calculate head-to-head statistics
print("Calculating head-to-head statistics...")
h2h_stats = calculate_h2h_stats(all_data)

print(f"H2H stats calculated: {len(h2h_stats)} records")
h2h_stats.head(10)

Calculating head-to-head statistics...
H2H stats calculated: 21526 records


,match_idx,date,winner,loser,h2h_total_meetings,h2h_winner_wins,h2h_loser_wins,h2h_winner_win_pct,h2h_last_meeting_date,h2h_days_since_last_meeting,h2h_last_meeting_winner_won,h2h_surface_meetings,h2h_surface_winner_wins,h2h_surface_winner_win_pct,h2h_winner_won_last_2
0,0,2016-01-04,Mikhail Kukushkin,Denis Istomin,0,0,0,NaN,NaT,NaN,NaN,0,0,NaN,NaN
1,62,2016-01-04,Daniel Munoz de la Nava,Feliciano Lopez,0,0,0,NaN,NaT,NaN,NaN,0,0,NaN,NaN
2,61,2016-01-04,Jeremy Chardy,Mubarak Shannan Zayid,0,0,0,NaN,NaT,NaN,NaN,0,0,NaN,NaN
3,60,2016-01-04,Paul Henri Mathieu,Albert Ramos,0,0,0,NaN,NaT,NaN,NaN,0,0,NaN,NaN
4,59,2016-01-04,Teymuraz Gabashvili,Simone Bolelli,0,0,0,NaN,NaT,NaN,NaN,0,0,NaN,NaN
5,58,2016-01-04,Illya Marchenko,David Ferrer,0,0,0,NaN,NaT,NaN,NaN,0,0,NaN,NaN
6,57,2016-01-04,Ricardas Berankis,Andreas Seppi,0,0,0,NaN,NaT,NaN,NaN,0,0,NaN,NaN
7,56,2016-01-04,Andrey Kuznetsov,Marsel Ilhan,0,0,0,NaN,NaT,NaN,NaN,0,0,NaN,NaN
8,55,2016-01-04,Robin Haase,Aslan Karatsev,0,0,0,NaN,NaT,NaN,NaN,0,0,NaN,NaN
9,53,2016-01-04,Stan Wawrinka,Borna Coric,0,0,0,NaN,NaT,NaN,NaN,0,0,NaN,NaN


## 5. Merge All Features

Combine all engineered features with the original match data.

In [16]:
# Add index to all_data for merging
all_data_indexed = all_data.reset_index(drop=False).rename(columns={'index': 'match_idx'})

# Merge winner form features
print("Merging winner form features...")
merged = all_data_indexed.merge(
    winner_form,
    left_on='match_idx',
    right_on='match_idx',
    how='left',
    suffixes=('', '_form')
)

# Merge loser form features
print("Merging loser form features...")
merged = merged.merge(
    loser_form,
    left_on='match_idx',
    right_on='match_idx',
    how='left',
    suffixes=('', '_form')
)

# Merge winner surface features
print("Merging winner surface features...")
merged = merged.merge(
    winner_surface,
    left_on='match_idx',
    right_on='match_idx',
    how='left',
    suffixes=('', '_surface')
)

# Merge loser surface features
print("Merging loser surface features...")
merged = merged.merge(
    loser_surface,
    left_on='match_idx',
    right_on='match_idx',
    how='left',
    suffixes=('', '_surface')
)

# Merge H2H features
print("Merging head-to-head features...")
merged = merged.merge(
    h2h_stats,
    left_on='match_idx',
    right_on='match_idx',
    how='left',
    suffixes=('', '_h2h')
)

# Remove duplicate columns
cols_to_drop = [col for col in merged.columns if col.endswith('_form') or 
                col.endswith('_surface') or col.endswith('_h2h') or 
                col in ['player', 'player_form', 'player_surface', 'date', 'date_form', 'date_surface']]
merged = merged.drop(columns=cols_to_drop, errors='ignore')

print(f"\nMerged dataset shape: {merged.shape}")
print(f"Total columns: {len(merged.columns)}")
print("\nNew feature columns added:")
new_cols = [col for col in merged.columns if col not in all_data.columns]
for col in sorted(new_cols):
    print(f"  - {col}")

Merging winner form features...
Merging loser form features...
Merging winner surface features...
Merging loser surface features...
Merging head-to-head features...

Merged dataset shape: (21526, 116)
Total columns: 116

New feature columns added:
  - h2h_days_since_last_meeting
  - h2h_last_meeting_date
  - h2h_last_meeting_winner_won
  - h2h_loser_wins
  - h2h_surface_meetings
  - h2h_surface_winner_win_pct
  - h2h_surface_winner_wins
  - h2h_total_meetings
  - h2h_winner_win_pct
  - h2h_winner_wins
  - h2h_winner_won_last_2
  - loser
  - loser_best_surface_win_rate
  - loser_career_matches
  - loser_career_win_rate
  - loser_career_wins
  - loser_carpet_differential
  - loser_carpet_matches
  - loser_carpet_win_rate
  - loser_carpet_wins
  - loser_clay_differential
  - loser_clay_matches
  - loser_clay_win_rate
  - loser_clay_wins
  - loser_grass_differential
  - loser_grass_matches
  - loser_grass_win_rate
  - loser_grass_wins
  - loser_hard_differential
  - loser_hard_matches
  - 

In [17]:
# Display sample of merged data with new features
print("Sample of merged data with engineered features:")
feature_cols = ['winner_name', 'loser_name', 'surface', 
                'winner_win_ratio_90d', 'loser_win_ratio_90d',
                'winner_hard_win_rate', 'loser_hard_win_rate',
                'h2h_total_meetings', 'h2h_winner_win_pct']
merged[feature_cols].head(10)

Sample of merged data with engineered features:


,winner_name,loser_name,surface,winner_win_ratio_90d,loser_win_ratio_90d,winner_hard_win_rate,loser_hard_win_rate,h2h_total_meetings,h2h_winner_win_pct
0,Mikhail Kukushkin,Denis Istomin,Hard,NaN,NaN,NaN,NaN,0,NaN
1,Radek Stepanek,Dusan Lajovic,Hard,NaN,NaN,NaN,NaN,0,NaN
2,Bernard Tomic,Nicolas Mahut,Hard,NaN,NaN,NaN,NaN,0,NaN
3,Ivan Dodig,Oliver Anderson,Hard,NaN,NaN,NaN,NaN,0,NaN
4,Lucas Pouille,Yoshihito Nishioka,Hard,NaN,NaN,NaN,NaN,0,NaN
5,David Goffin,Thomaz Bellucci,Hard,NaN,NaN,NaN,NaN,0,NaN
6,Dominic Thiem,James Duckworth,Hard,NaN,NaN,NaN,NaN,0,NaN
7,Denis Kudla,John Patrick Smith,Hard,NaN,NaN,NaN,NaN,0,NaN
8,Hyeon Chung,Sam Groth,Hard,NaN,NaN,NaN,NaN,0,NaN
9,Grigor Dimitrov,Gilles Simon,Hard,NaN,NaN,NaN,NaN,0,NaN


## 6. Feature Summary Statistics

Analyze the distribution and coverage of our engineered features.

In [18]:
# Summary statistics for recent form features
form_features = ['winner_win_ratio_90d', 'winner_win_ratio_180d', 'winner_win_ratio_365d',
                 'loser_win_ratio_90d', 'loser_win_ratio_180d', 'loser_win_ratio_365d']

print("=== RECENT FORM FEATURES SUMMARY ===\n")
print(merged[form_features].describe())
print(f"\nMissing values:")
print(merged[form_features].isnull().sum())

=== RECENT FORM FEATURES SUMMARY ===

       winner_win_ratio_90d  winner_win_ratio_180d  winner_win_ratio_365d  \
count          19787.000000           20376.000000           20701.000000   
mean               0.512269               0.526523               0.531232   
std                0.225285               0.195815               0.178733   
min                0.000000               0.000000               0.000000   
25%                0.400000               0.428571               0.440000   
50%                0.533333               0.538462               0.531915   
75%                0.666667               0.650000               0.644444   
max                1.000000               1.000000               1.000000   

       loser_win_ratio_90d  loser_win_ratio_180d  loser_win_ratio_365d  
count         18598.000000          19553.000000          20087.000000  
mean              0.440068              0.458841              0.467219  
std               0.226897              0.196753 

In [19]:
# Summary statistics for surface features
surface_features = ['winner_hard_win_rate', 'winner_clay_win_rate', 'winner_grass_win_rate',
                    'loser_hard_win_rate', 'loser_clay_win_rate', 'loser_grass_win_rate',
                    'winner_surface_versatility', 'loser_surface_versatility']

print("\n=== SURFACE SPECIALIZATION FEATURES SUMMARY ===\n")
print(merged[surface_features].describe())
print(f"\nMissing values:")
print(merged[surface_features].isnull().sum())


=== SURFACE SPECIALIZATION FEATURES SUMMARY ===

       winner_hard_win_rate  winner_clay_win_rate  winner_grass_win_rate  \
count          20733.000000          19684.000000           17984.000000   
mean               0.510228              0.503371               0.464729   
std                0.181805              0.191523               0.243778   
min                0.000000              0.000000               0.000000   
25%                0.416667              0.411765               0.333333   
50%                0.514620              0.521739               0.500000   
75%                0.625000              0.615385               0.642857   
max                1.000000              1.000000               1.000000   

       loser_hard_win_rate  loser_clay_win_rate  loser_grass_win_rate  \
count         20052.000000         18768.000000          16751.000000   
mean              0.448931             0.453348              0.407940   
std               0.179017             0.19147

In [20]:
# Summary statistics for H2H features
h2h_features = ['h2h_total_meetings', 'h2h_winner_win_pct', 'h2h_days_since_last_meeting',
                'h2h_surface_winner_win_pct']

print("\n=== HEAD-TO-HEAD FEATURES SUMMARY ===\n")
print(merged[h2h_features].describe())
print(f"\nMissing values:")
print(merged[h2h_features].isnull().sum())

# Count matches with prior H2H history
print(f"\nMatches with prior H2H history: {(merged['h2h_total_meetings'] > 0).sum()} "
      f"({(merged['h2h_total_meetings'] > 0).sum() / len(merged) * 100:.1f}%)")


=== HEAD-TO-HEAD FEATURES SUMMARY ===

       h2h_total_meetings  h2h_winner_win_pct  h2h_days_since_last_meeting  \
count        21526.000000         7588.000000                  7588.000000   
mean             0.673929            0.583403                   423.752372   
std              1.260723            0.430131                   441.320621   
min              0.000000            0.000000                     3.000000   
25%              0.000000            0.000000                   112.000000   
50%              0.000000            0.666667                   273.000000   
75%              1.000000            1.000000                   595.000000   
max             16.000000            1.000000                  3045.000000   

       h2h_surface_winner_win_pct  
count                 4842.000000  
mean                     0.589991  
std                      0.447885  
min                      0.000000  
25%                      0.000000  
50%                      0.800000  
75%  

## 7. Export Engineered Features

Save the engineered features for each year to Excel files.

In [21]:
# Create output directory
output_folder = r'c:\poly_code\COMP. DATA ANALYSIS PROJECT\Featured_Data_Atp_matches'
os.makedirs(output_folder, exist_ok=True)

print(f"Output folder created: {output_folder}")

# Extract year from tourney_date for splitting
merged['year'] = pd.to_datetime(merged['tourney_date']).dt.year

print(f"\nYears in dataset: {sorted(merged['year'].unique())}")

Output folder created: c:\poly_code\COMP. DATA ANALYSIS PROJECT\Featured_Data_Atp_matches

Years in dataset: [np.int32(2016), np.int32(2017), np.int32(2018), np.int32(2019), np.int32(2021), np.int32(2022), np.int32(2023), np.int32(2024)]


In [22]:
# Save data for each year
for year in sorted(merged['year'].unique()):
    year_data = merged[merged['year'] == year].copy()
    
    # Drop temporary columns
    year_data = year_data.drop(columns=['match_idx', 'year'], errors='ignore')
    
    # Save to Excel
    output_file = os.path.join(output_folder, f'featured_matches_{int(year)}.xlsx')
    year_data.to_excel(output_file, index=False, engine='openpyxl')
    
    print(f"Saved {year}: {len(year_data)} matches -> {output_file}")

print(f"\n✓ All files saved to: {output_folder}")

Saved 2016: 2812 matches -> c:\poly_code\COMP. DATA ANALYSIS PROJECT\Featured_Data_Atp_matches\featured_matches_2016.xlsx
Saved 2017: 2744 matches -> c:\poly_code\COMP. DATA ANALYSIS PROJECT\Featured_Data_Atp_matches\featured_matches_2017.xlsx
Saved 2018: 2868 matches -> c:\poly_code\COMP. DATA ANALYSIS PROJECT\Featured_Data_Atp_matches\featured_matches_2018.xlsx
Saved 2019: 2549 matches -> c:\poly_code\COMP. DATA ANALYSIS PROJECT\Featured_Data_Atp_matches\featured_matches_2019.xlsx
Saved 2021: 2573 matches -> c:\poly_code\COMP. DATA ANALYSIS PROJECT\Featured_Data_Atp_matches\featured_matches_2021.xlsx
Saved 2022: 2558 matches -> c:\poly_code\COMP. DATA ANALYSIS PROJECT\Featured_Data_Atp_matches\featured_matches_2022.xlsx
Saved 2023: 2668 matches -> c:\poly_code\COMP. DATA ANALYSIS PROJECT\Featured_Data_Atp_matches\featured_matches_2023.xlsx
Saved 2024: 2754 matches -> c:\poly_code\COMP. DATA ANALYSIS PROJECT\Featured_Data_Atp_matches\featured_matches_2024.xlsx

✓ All files saved to: c

## 8. Feature Engineering Summary

Summary of all features created and their purpose for upset prediction.

In [23]:
print("=" * 80)
print("FEATURE ENGINEERING SUMMARY")
print("=" * 80)

print("\n1. RECENT FORM FEATURES (for both winner and loser):")
print("   - win_ratio_90d: Win-loss ratio over past 3 months")
print("   - win_ratio_180d: Win-loss ratio over past 6 months")
print("   - win_ratio_365d: Win-loss ratio over past 12 months")
print("   - wins_XXd, losses_XXd, total_matches_XXd: Supporting statistics")

print("\n2. SURFACE SPECIALIZATION FEATURES (for both winner and loser):")
print("   - hard/clay/grass/carpet_win_rate: Win rate on each surface")
print("   - hard/clay/grass/carpet_differential: Surface performance vs overall")
print("   - hard/clay/grass/carpet_matches: Experience on each surface")
print("   - best_surface, worst_surface: Strongest and weakest surfaces")
print("   - surface_versatility: Difference between best and worst surface rates")
print("   - career_win_rate: Overall career win percentage")

print("\n3. HEAD-TO-HEAD FEATURES:")
print("   - h2h_total_meetings: Previous matches between these opponents")
print("   - h2h_winner_win_pct: Winner's historical win % against loser")
print("   - h2h_winner_wins, h2h_loser_wins: Head-to-head record")
print("   - h2h_days_since_last_meeting: Days since they last played")
print("   - h2h_last_meeting_winner_won: Whether winner won last meeting")
print("   - h2h_surface_winner_win_pct: H2H win % on current surface")
print("   - h2h_winner_won_last_2: Whether winner won last 2 meetings")

print("\n4. USE CASES FOR UPSET PREDICTION:")
print("   - Compare form differentials (winner vs loser recent win rates)")
print("   - Identify surface mismatches (e.g., clay specialist on grass)")
print("   - Detect psychological edges (dominant H2H records)")
print("   - Spot momentum shifts (recent form trends)")
print("   - Account for rust or fatigue (days since last match)")

print("\n" + "=" * 80)
print(f"Total engineered features: {len([col for col in merged.columns if any(x in col for x in ['win_ratio', 'surface', 'h2h', 'career'])])}")
print(f"Total columns in final dataset: {len(merged.columns) - 2}")  # Excluding match_idx and year
print("=" * 80)

FEATURE ENGINEERING SUMMARY

1. RECENT FORM FEATURES (for both winner and loser):
   - win_ratio_90d: Win-loss ratio over past 3 months
   - win_ratio_180d: Win-loss ratio over past 6 months
   - win_ratio_365d: Win-loss ratio over past 12 months
   - wins_XXd, losses_XXd, total_matches_XXd: Supporting statistics

2. SURFACE SPECIALIZATION FEATURES (for both winner and loser):
   - hard/clay/grass/carpet_win_rate: Win rate on each surface
   - hard/clay/grass/carpet_differential: Surface performance vs overall
   - hard/clay/grass/carpet_matches: Experience on each surface
   - best_surface, worst_surface: Strongest and weakest surfaces
   - surface_versatility: Difference between best and worst surface rates
   - career_win_rate: Overall career win percentage

3. HEAD-TO-HEAD FEATURES:
   - h2h_total_meetings: Previous matches between these opponents
   - h2h_winner_win_pct: Winner's historical win % against loser
   - h2h_winner_wins, h2h_loser_wins: Head-to-head record
   - h2h_days

## ✅ Feature Engineering Complete!

All engineered features have been successfully calculated and exported to:
`Featured_Data_Atp_matches/`

### Files Created:
- featured_matches_2016.xlsx (2,812 matches)
- featured_matches_2017.xlsx (2,744 matches)
- featured_matches_2018.xlsx (2,868 matches)
- featured_matches_2019.xlsx (2,549 matches)
- featured_matches_2021.xlsx (2,573 matches)
- featured_matches_2022.xlsx (2,558 matches)
- featured_matches_2023.xlsx (2,668 matches)
- featured_matches_2024.xlsx (2,754 matches)

### Key Insights:
- **82 new features** created for upset prediction analysis
- **Recent Form**: Winners have better recent form (avg 51-53% win rate) vs losers (44-47%)
- **Surface Versatility**: Average versatility (diff between best/worst surface) ~18% for both winners and losers
- **H2H History**: 35.3% of matches have prior head-to-head history, with winners typically dominant (58% H2H win rate)

These features can now be used for machine learning models to predict tennis match upsets!

In [24]:
# Quick verification - show a few matches with all feature types
print("Sample matches with all engineered features:\n")

sample_cols = [
    'tourney_name', 'winner_name', 'loser_name', 'surface',
    'winner_win_ratio_90d', 'loser_win_ratio_90d',  # Recent form
    'winner_hard_win_rate', 'loser_hard_win_rate',  # Surface specialization
    'h2h_total_meetings', 'h2h_winner_win_pct',     # H2H
    'upset'  # Target variable
]

# Show some matches with H2H history
matches_with_h2h = merged[merged['h2h_total_meetings'] > 0].head(10)
print(matches_with_h2h[sample_cols])

Sample matches with all engineered features:

        tourney_name            winner_name             loser_name surface  \
102         Auckland            Lukas Rosol           Benoit Paire    Hard   
133           Sydney         Viktor Troicki        Grigor Dimitrov    Hard   
170  Australian Open           Milos Raonic          Lucas Pouille    Hard   
228  Australian Open          Roger Federer        Grigor Dimitrov    Hard   
344     Buenos Aires          Paolo Lorenzi          Pablo Andujar    Clay   
360     Buenos Aires          Dominic Thiem        Nicolas Almagro    Clay   
383          Memphis          Kei Nishikori      Mikhail Kukushkin    Hard   
407        Rotterdam  Roberto Bautista Agut            Jiri Vesely    Hard   
413        Rotterdam          Martin Klizan  Roberto Bautista Agut    Hard   
414        Rotterdam          Nicolas Mahut         Viktor Troicki    Hard   

     winner_win_ratio_90d  loser_win_ratio_90d  winner_hard_win_rate  \
102              0.5000

## Next Steps: Using These Features for Upset Prediction

### Suggested Analysis Approaches:

1. **Form Differential Analysis**
   - Calculate `form_diff = winner_win_ratio_90d - loser_win_ratio_90d`
   - Analyze how form differentials relate to upset probability
   - Smaller differentials → higher upset probability

2. **Surface Mismatch Detection**
   - Compare surface-specific win rates for current match surface
   - Example: Clay specialist playing on grass = upset risk
   - Feature: `surface_advantage = winner_[surface]_win_rate - loser_[surface]_win_rate`

3. **H2H Psychological Factors**
   - Players with dominant H2H records have psychological edge
   - Recent H2H results (`h2h_last_meeting_winner_won`) matter
   - Feature importance: `h2h_winner_win_pct`, `h2h_winner_won_last_2`

4. **Model Building**
   - Target variable: `upset` (already in dataset)
   - Features: All engineered features (82 total)
   - Consider: Random Forest, XGBoost, Neural Networks
   - Handle missing values appropriately (early career players)

5. **Feature Interactions**
   - `form_momentum = (winner_win_ratio_90d - winner_win_ratio_365d)`
   - `surface_comfort = winner_[current_surface]_win_rate - winner_career_win_rate`
   - `experience_gap = winner_career_matches - loser_career_matches`

## 9. Create Combined Features & Remove Redundant Ones

Combine raw features into more meaningful derived features that better capture upset dynamics.

In [25]:
# Create a copy for feature engineering
featured_data = merged.copy()

print("Creating combined features for upset prediction...")
print(f"Starting with {featured_data.shape[1]} columns\n")

# 1. FORM DIFFERENTIALS (Winner advantage over Loser)
featured_data['form_diff_90d'] = featured_data['winner_win_ratio_90d'] - featured_data['loser_win_ratio_90d']
featured_data['form_diff_180d'] = featured_data['winner_win_ratio_180d'] - featured_data['loser_win_ratio_180d']
featured_data['form_diff_365d'] = featured_data['winner_win_ratio_365d'] - featured_data['loser_win_ratio_365d']

# 2. FORM MOMENTUM (Recent vs Long-term form)
featured_data['winner_form_momentum'] = featured_data['winner_win_ratio_90d'] - featured_data['winner_win_ratio_365d']
featured_data['loser_form_momentum'] = featured_data['loser_win_ratio_90d'] - featured_data['loser_win_ratio_365d']
featured_data['momentum_diff'] = featured_data['winner_form_momentum'] - featured_data['loser_form_momentum']

# 3. SURFACE-SPECIFIC ADVANTAGES (for current match surface)
# Get current surface win rates for both players
def get_surface_advantage(row):
    surface = row['surface'].lower()
    winner_rate = row.get(f'winner_{surface}_win_rate', np.nan)
    loser_rate = row.get(f'loser_{surface}_win_rate', np.nan)
    if pd.notna(winner_rate) and pd.notna(loser_rate):
        return winner_rate - loser_rate
    return np.nan

featured_data['surface_advantage'] = featured_data.apply(get_surface_advantage, axis=1)

# 4. SURFACE COMFORT (How well each player performs on current surface vs their overall)
def get_surface_comfort(row, player='winner'):
    surface = row['surface'].lower()
    surface_rate = row.get(f'{player}_{surface}_win_rate', np.nan)
    career_rate = row.get(f'{player}_career_win_rate', np.nan)
    if pd.notna(surface_rate) and pd.notna(career_rate):
        return surface_rate - career_rate
    return np.nan

featured_data['winner_surface_comfort'] = featured_data.apply(lambda x: get_surface_comfort(x, 'winner'), axis=1)
featured_data['loser_surface_comfort'] = featured_data.apply(lambda x: get_surface_comfort(x, 'loser'), axis=1)
featured_data['surface_comfort_diff'] = featured_data['winner_surface_comfort'] - featured_data['loser_surface_comfort']

# 5. EXPERIENCE GAP
featured_data['experience_gap'] = featured_data['winner_career_matches'] - featured_data['loser_career_matches']
featured_data['experience_gap_pct'] = (featured_data['experience_gap'] / 
                                       (featured_data['winner_career_matches'] + featured_data['loser_career_matches'] + 1))

# 6. CAREER WIN RATE DIFFERENTIAL
featured_data['career_win_rate_diff'] = featured_data['winner_career_win_rate'] - featured_data['loser_career_win_rate']

# 7. SURFACE VERSATILITY COMPARISON
featured_data['versatility_diff'] = featured_data['winner_surface_versatility'] - featured_data['loser_surface_versatility']

# 8. H2H DOMINANCE (simplified)
featured_data['h2h_dominance'] = featured_data['h2h_winner_win_pct'].fillna(0.5)  # neutral if no history
featured_data['h2h_recent_momentum'] = featured_data['h2h_last_meeting_winner_won'].fillna(0.5)

# 9. RECENT ACTIVITY (matches played in last 90 days)
featured_data['winner_recent_activity'] = featured_data['winner_total_matches_90d']
featured_data['loser_recent_activity'] = featured_data['loser_total_matches_90d']
featured_data['activity_diff'] = featured_data['winner_recent_activity'] - featured_data['loser_recent_activity']

print("✓ Combined features created!")
print(f"\nNew combined features:")
combined_features = ['form_diff_90d', 'form_diff_180d', 'form_diff_365d', 
                     'winner_form_momentum', 'loser_form_momentum', 'momentum_diff',
                     'surface_advantage', 'winner_surface_comfort', 'loser_surface_comfort', 
                     'surface_comfort_diff', 'experience_gap', 'experience_gap_pct',
                     'career_win_rate_diff', 'versatility_diff', 'h2h_dominance', 
                     'h2h_recent_momentum', 'winner_recent_activity', 'loser_recent_activity', 
                     'activity_diff']
for feat in combined_features:
    print(f"  - {feat}")

Creating combined features for upset prediction...
Starting with 117 columns

✓ Combined features created!

New combined features:
  - form_diff_90d
  - form_diff_180d
  - form_diff_365d
  - winner_form_momentum
  - loser_form_momentum
  - momentum_diff
  - surface_advantage
  - winner_surface_comfort
  - loser_surface_comfort
  - surface_comfort_diff
  - experience_gap
  - experience_gap_pct
  - career_win_rate_diff
  - versatility_diff
  - h2h_dominance
  - h2h_recent_momentum
  - winner_recent_activity
  - loser_recent_activity
  - activity_diff


In [26]:
# Identify and remove redundant/less important features
print("\nRemoving redundant features...")

# Features to remove (keeping only the most useful ones)
features_to_remove = [
    # Raw win/loss counts (keeping only ratios)
    'winner_wins_90d', 'winner_losses_90d', 'winner_wins_180d', 'winner_losses_180d',
    'winner_wins_365d', 'winner_losses_365d', 'loser_wins_90d', 'loser_losses_90d',
    'loser_wins_180d', 'loser_losses_180d', 'loser_wins_365d', 'loser_losses_365d',
    
    # Raw match counts for individual time periods (keeping total career matches)
    'winner_total_matches_180d', 'winner_total_matches_365d',
    'loser_total_matches_180d', 'loser_total_matches_365d',
    
    # Individual surface match counts and wins (keeping only win rates and differentials)
    'winner_hard_matches', 'winner_hard_wins', 'winner_clay_matches', 'winner_clay_wins',
    'winner_grass_matches', 'winner_grass_wins', 'winner_carpet_matches', 'winner_carpet_wins',
    'loser_hard_matches', 'loser_hard_wins', 'loser_clay_matches', 'loser_clay_wins',
    'loser_grass_matches', 'loser_grass_wins', 'loser_carpet_matches', 'loser_carpet_wins',
    
    # Carpet surface features (very rare, mostly NaN)
    'winner_carpet_win_rate', 'winner_carpet_differential', 
    'loser_carpet_win_rate', 'loser_carpet_differential',
    
    # Individual career wins (keeping only win rates)
    'winner_career_wins', 'loser_career_wins',
    
    # H2H individual wins (keeping only percentages and dominance metrics)
    'h2h_winner_wins', 'h2h_loser_wins', 'h2h_surface_winner_wins',
    
    # Redundant text columns
    'winner', 'loser', 'winner_current_surface', 'loser_current_surface',
    
    # Best/worst surface names (keeping versatility metric instead)
    'winner_best_surface', 'winner_worst_surface', 'loser_best_surface', 'loser_worst_surface',
    'winner_best_surface_win_rate', 'winner_worst_surface_win_rate',
    'loser_best_surface_win_rate', 'loser_worst_surface_win_rate',
    
    # H2H last meeting date (keeping days since instead)
    'h2h_last_meeting_date',
    
    # Redundant individual surface win ratios (using combined features instead)
    'winner_win_ratio_180d', 'winner_win_ratio_365d',
    'loser_win_ratio_180d', 'loser_win_ratio_365d',
    
    # Temporary index column
    'match_idx'
]

# Remove features that exist in the dataframe
features_removed = [f for f in features_to_remove if f in featured_data.columns]
featured_data = featured_data.drop(columns=features_removed)

print(f"✓ Removed {len(features_removed)} redundant features")
print(f"✓ Final dataset: {featured_data.shape[1]} columns\n")

# Show reduction
print(f"Feature reduction: {merged.shape[1]} → {featured_data.shape[1]} columns")


Removing redundant features...
✓ Removed 53 redundant features
✓ Final dataset: 83 columns

Feature reduction: 117 → 83 columns


In [27]:
# Display summary of key features for upset prediction
print("=" * 80)
print("KEY FEATURES FOR UPSET PREDICTION")
print("=" * 80)

print("\n1. FORM-BASED FEATURES:")
form_cols = ['form_diff_90d', 'winner_win_ratio_90d', 'loser_win_ratio_90d',
             'winner_form_momentum', 'loser_form_momentum', 'momentum_diff']
print(featured_data[form_cols].describe())

print("\n2. SURFACE-BASED FEATURES:")
surface_cols = ['surface_advantage', 'surface_comfort_diff', 
                'winner_surface_comfort', 'loser_surface_comfort', 'versatility_diff']
print(featured_data[surface_cols].describe())

print("\n3. EXPERIENCE & CAREER FEATURES:")
exp_cols = ['experience_gap', 'experience_gap_pct', 'career_win_rate_diff',
            'winner_career_matches', 'loser_career_matches']
print(featured_data[exp_cols].describe())

print("\n4. HEAD-TO-HEAD FEATURES:")
h2h_cols = ['h2h_total_meetings', 'h2h_dominance', 'h2h_recent_momentum',
            'h2h_days_since_last_meeting']
print(featured_data[h2h_cols].describe())

KEY FEATURES FOR UPSET PREDICTION

1. FORM-BASED FEATURES:
       form_diff_90d  winner_win_ratio_90d  loser_win_ratio_90d  \
count   17692.000000          19787.000000         18598.000000   
mean        0.077135              0.512269             0.440068   
std         0.294516              0.225285             0.226897   
min        -1.000000              0.000000             0.000000   
25%        -0.100000              0.400000             0.333333   
50%         0.071429              0.533333             0.500000   
75%         0.259740              0.666667             0.600000   
max         1.000000              1.000000             1.000000   

       winner_form_momentum  loser_form_momentum  momentum_diff  
count          19787.000000         18598.000000   17692.000000  
mean              -0.027069            -0.039018       0.012411  
std                0.145283             0.156315       0.207098  
min               -0.888889            -0.750000      -1.035714  
25%    

In [28]:
# Analyze correlation of combined features with upsets
print("\n" + "=" * 80)
print("CORRELATION WITH UPSETS")
print("=" * 80)

# Calculate correlations with upset column
key_features = ['form_diff_90d', 'momentum_diff', 'surface_advantage', 'surface_comfort_diff',
                'experience_gap_pct', 'career_win_rate_diff', 'h2h_dominance', 
                'versatility_diff', 'activity_diff']

correlations = []
for feat in key_features:
    if feat in featured_data.columns and 'upset' in featured_data.columns:
        corr = featured_data[[feat, 'upset']].corr().iloc[0, 1]
        correlations.append({'Feature': feat, 'Correlation': corr})

corr_df = pd.DataFrame(correlations).sort_values('Correlation', ascending=False)
print("\nFeature correlations with upset (negative = favors favorite winning):")
print(corr_df.to_string(index=False))


CORRELATION WITH UPSETS

Feature correlations with upset (negative = favors favorite winning):
             Feature  Correlation
surface_comfort_diff     0.005297
       momentum_diff    -0.009582
    versatility_diff    -0.061129
       h2h_dominance    -0.130001
   surface_advantage    -0.334777
       form_diff_90d    -0.346103
career_win_rate_diff    -0.419678
       activity_diff    -0.466437
  experience_gap_pct    -0.466715


## 10. Export Cleaned Featured Data

In [29]:
# Save the cleaned, combined features dataset
output_folder_combined = r'c:\poly_code\COMP. DATA ANALYSIS PROJECT\Featured_Data_Atp_matches'

print("Saving cleaned combined features to Excel files...")

# Extract year for splitting
featured_data['year'] = pd.to_datetime(featured_data['tourney_date']).dt.year

# Save each year
for year in sorted(featured_data['year'].unique()):
    year_data = featured_data[featured_data['year'] == year].copy()
    year_data = year_data.drop(columns=['year'], errors='ignore')
    
    output_file = os.path.join(output_folder_combined, f'featured_matches_{int(year)}.xlsx')
    year_data.to_excel(output_file, index=False, engine='openpyxl')
    
    print(f"✓ Saved {year}: {len(year_data)} matches, {len(year_data.columns)} features -> {output_file}")

print(f"\n✓ All combined feature files saved to: {output_folder_combined}")
print(f"\n{'='*80}")
print("FINAL DATASET SUMMARY")
print(f"{'='*80}")
print(f"Total matches: {len(featured_data):,}")
print(f"Total features: {len(featured_data.columns)}")
print(f"Feature reduction: {merged.shape[1]} → {featured_data.shape[1]} ({merged.shape[1] - featured_data.shape[1]} removed)")
print(f"Combined features added: {len(combined_features)}")
print(f"{'='*80}")

Saving cleaned combined features to Excel files...
✓ Saved 2016: 2812 matches, 82 features -> c:\poly_code\COMP. DATA ANALYSIS PROJECT\Featured_Data_Atp_matches\featured_matches_2016.xlsx
✓ Saved 2017: 2744 matches, 82 features -> c:\poly_code\COMP. DATA ANALYSIS PROJECT\Featured_Data_Atp_matches\featured_matches_2017.xlsx
✓ Saved 2018: 2868 matches, 82 features -> c:\poly_code\COMP. DATA ANALYSIS PROJECT\Featured_Data_Atp_matches\featured_matches_2018.xlsx
✓ Saved 2019: 2549 matches, 82 features -> c:\poly_code\COMP. DATA ANALYSIS PROJECT\Featured_Data_Atp_matches\featured_matches_2019.xlsx
✓ Saved 2021: 2573 matches, 82 features -> c:\poly_code\COMP. DATA ANALYSIS PROJECT\Featured_Data_Atp_matches\featured_matches_2021.xlsx
✓ Saved 2022: 2558 matches, 82 features -> c:\poly_code\COMP. DATA ANALYSIS PROJECT\Featured_Data_Atp_matches\featured_matches_2022.xlsx
✓ Saved 2023: 2668 matches, 82 features -> c:\poly_code\COMP. DATA ANALYSIS PROJECT\Featured_Data_Atp_matches\featured_matches_

## 🎯 Combined Features Summary

### Top Predictive Features (by correlation with upsets):

1. **experience_gap_pct** (-0.467): Experience difference matters most for upsets
2. **activity_diff** (-0.466): Recent activity level differential
3. **career_win_rate_diff** (-0.420): Career performance gap
4. **form_diff_90d** (-0.346): Recent 3-month form differential
5. **surface_advantage** (-0.335): Surface-specific skill advantage
6. **h2h_dominance** (-0.130): Head-to-head psychological edge

**Negative correlation** = Lower values → Higher upset probability

### Key Insights:

- **Experience & Activity** are the strongest upset indicators
  - Less experienced players with recent matches can cause upsets
  
- **Form differentials** matter, especially recent form (90d)
  - Players in good recent form can overcome career statistics
  
- **Surface advantage** is critical
  - A clay specialist on hard court is vulnerable to upsets
  
- **H2H history** provides psychological context
  - But it's less predictive than form and experience

### Removed Features (53 total):
- Raw counts (wins/losses) → Kept only ratios
- Individual surface match counts → Kept only win rates
- Carpet surface features → Too rare, mostly NaN
- Redundant text columns → Kept only essential identifiers
- Long-term form features → Kept only 90d recent form
- Best/worst surface names → Kept versatility metric instead

In [30]:
# Show examples of matches with combined features
print("SAMPLE MATCHES WITH COMBINED FEATURES:\n")

display_cols = ['winner_name', 'loser_name', 'surface', 'upset',
                'form_diff_90d', 'surface_advantage', 'experience_gap_pct',
                'career_win_rate_diff', 'h2h_dominance']

# Show some upset matches
upsets = featured_data[featured_data['upset'] == 1].head(5)
print("=== UPSET MATCHES (Lower-ranked player won) ===")
print(upsets[display_cols].to_string(index=False))

print("\n=== EXPECTED WINS (Favorite won) ===")
# Show some non-upset matches
expected = featured_data[featured_data['upset'] == 0].head(5)
print(expected[display_cols].to_string(index=False))

print("\n" + "="*80)
print("INTERPRETATION:")
print("="*80)
print("• Upsets typically have:")
print("  - NEGATIVE form_diff_90d (underdog in better recent form)")
print("  - NEGATIVE surface_advantage (favorite uncomfortable on surface)")
print("  - NEGATIVE experience_gap_pct (underdog has more experience)")
print("  - Lower career_win_rate_diff (smaller skill gap)")
print("  - h2h_dominance near 0.5 (no psychological edge)")
print("="*80)

SAMPLE MATCHES WITH COMBINED FEATURES:

=== UPSET MATCHES (Lower-ranked player won) ===
        winner_name           loser_name surface  upset  form_diff_90d  surface_advantage  experience_gap_pct  career_win_rate_diff  h2h_dominance
     Radek Stepanek        Dusan Lajovic    Hard      1            NaN                NaN                 0.0                   NaN            0.5
       Tobias Kamke    Benjamin Mitchell    Hard      1            NaN                NaN                 0.0                   NaN            0.5
      Lucas Pouille         David Goffin    Hard      1            NaN                NaN                 0.0                   NaN            0.5
Ramkumar Ramanathan Daniel Gimeno Traver    Hard      1            NaN                NaN                 0.0                   NaN            0.5
         Ante Pavic      Nicolas Almagro    Hard      1            NaN                NaN                 0.0                   NaN            0.5

=== EXPECTED WINS (Favorite w

In [31]:
# Display final feature list organized by category
print("\n" + "="*80)
print("FINAL FEATURE SET (83 features)")
print("="*80)

# Organize features by category
feature_categories = {
    "Match Context (8)": [col for col in featured_data.columns if col in 
                          ['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level', 
                           'tourney_date', 'winner_name', 'loser_name']],
    
    "Match Details (10)": [col for col in featured_data.columns if col in 
                           ['winner_seed', 'loser_seed', 'score', 'best_of', 'round', 
                            'minutes', 'rank_difference', 'upset', 'winner_age', 'loser_age']],
    
    "Player Physical (4)": [col for col in featured_data.columns if col in 
                            ['winner_ht', 'winner_ioc', 'loser_ht', 'loser_ioc']],
    
    "Serve Statistics (10)": [col for col in featured_data.columns if any(x in col for x in 
                               ['ace_df_ratio', '1st_serve', '2nd_serve', 'bp_saved', 'serve_efficiency'])],
    
    "Combined Form Features (9)": [col for col in featured_data.columns if any(x in col for x in 
                                    ['form_diff', 'form_momentum', 'momentum_diff', 'win_ratio_90d'])],
    
    "Combined Surface Features (10)": [col for col in featured_data.columns if any(x in col for x in 
                                        ['surface_advantage', 'surface_comfort', 'hard_win_rate', 
                                         'clay_win_rate', 'grass_win_rate', 'hard_differential', 
                                         'clay_differential', 'grass_differential', 'versatility'])],
    
    "Combined Experience Features (5)": [col for col in featured_data.columns if any(x in col for x in 
                                         ['experience_gap', 'career_matches', 'career_win_rate'])],
    
    "Combined H2H Features (6)": [col for col in featured_data.columns if 'h2h' in col],
    
    "Combined Activity Features (3)": [col for col in featured_data.columns if 'activity' in col],
}

for category, features in feature_categories.items():
    print(f"\n{category}:")
    for feat in sorted(features):
        print(f"  • {feat}")

print("\n" + "="*80)
print(f"Total: {sum(len(v) for v in feature_categories.values())} features organized")
print("="*80)


FINAL FEATURE SET (83 features)

Match Context (8):
  • draw_size
  • loser_name
  • surface
  • tourney_date
  • tourney_id
  • tourney_level
  • tourney_name
  • winner_name

Match Details (10):
  • best_of
  • loser_age
  • loser_seed
  • minutes
  • rank_difference
  • round
  • score
  • upset
  • winner_age
  • winner_seed

Player Physical (4):
  • loser_ht
  • loser_ioc
  • winner_ht
  • winner_ioc

Serve Statistics (10):
  • l_1st_serve_pct
  • l_1st_serve_won_pct
  • l_2nd_serve_won_pct
  • l_ace_df_ratio
  • l_bp_saved_pct
  • l_serve_efficiency
  • w_1st_serve_pct
  • w_1st_serve_won_pct
  • w_2nd_serve_won_pct
  • w_ace_df_ratio
  • w_bp_saved_pct
  • w_serve_efficiency

Combined Form Features (9):
  • form_diff_180d
  • form_diff_365d
  • form_diff_90d
  • loser_form_momentum
  • loser_win_ratio_90d
  • momentum_diff
  • winner_form_momentum
  • winner_win_ratio_90d

Combined Surface Features (10):
  • loser_clay_differential
  • loser_clay_win_rate
  • loser_grass_differ